In [1]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2, MobileNetV3Small, InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

2024-10-24 14:35:41.816668: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-24 14:35:41.819775: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-24 14:35:41.829671: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-24 14:35:41.846304: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-24 14:35:41.851056: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-24 14:35:41.863547: I tensorflow/core/platform/cpu_feature_gu

In [2]:
src_path = './classes'
num_classes = 5

train_data = tf.keras.preprocessing.image_dataset_from_directory(
    src_path,
    image_size=(224, 224),
    batch_size=32,
    label_mode='categorical',
    subset='training',
    validation_split=0.2,
    seed=123,
)

val_data = tf.keras.preprocessing.image_dataset_from_directory(
    src_path,
    image_size=(224, 224),
    batch_size=32,
    label_mode='categorical',
    subset='validation',
    validation_split=0.2,
    seed=123
)

Found 702 files belonging to 5 classes.
Using 562 files for training.
Found 702 files belonging to 5 classes.
Using 140 files for validation.


In [3]:
# Data augmentation layers
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal_and_vertical"),   # Flip images randomly
    tf.keras.layers.RandomRotation(0.1),                     # Rotate images by up to 20%
    tf.keras.layers.RandomZoom(0.1),                         # Zoom in by up to 20%
    tf.keras.layers.RandomContrast(0.1)                      # Change contrast by up to 20%
])

# Augment only the training data
# train_data = train_data.map( lambda x, y: (data_augmentation(x, training=True), y) )

In [5]:
AUTOTUNE = tf.data.AUTOTUNE

train_data = train_data.cache().prefetch(buffer_size=AUTOTUNE)
val_data   = val_data.cache().prefetch(buffer_size=AUTOTUNE)

base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

x = base_model.output
x = GlobalAveragePooling2D()(x)

# x = Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001))(x)

# reg = tf.keras.regularizers.l1_l2(l1=0.001, l2=0.001)
# x = Dense(256, activation='relu', kernel_regularizer=reg)(x)

# x = Dropout(0.5)(x)  # Dropout 50%
# x = Dropout(0.6)(x)


x = Dense(1024, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)


87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 15s 0us/step


In [7]:
from sklearn.utils import class_weight
import numpy as np

labels = [y.numpy().argmax(axis=1) for _, y in train_data]

class_labels = np.concatenate(labels, axis=0)

class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(class_labels), y=class_labels)

class_weights_dict = dict(enumerate(class_weights))

2024-10-24 14:43:00.536916: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [8]:
model = Model(inputs=base_model.input, outputs=predictions)

In [9]:
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.summary()

In [10]:
model.fit(train_data, epochs=10, validation_data=val_data)

Epoch 1/10
